In [6]:

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

In [23]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from src.tools.preprocess import preprocess_data
import rasterio
from skimage import exposure
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
import os
from PIL import Image
import pandas as pd

In [64]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
data = pd.read_csv('../data/processed/data.csv')
data_clean = preprocess_data(data[data["dataset"] == "train"])
data_clean.head()

In [45]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
import rasterio
from PIL import Image
from skimage import exposure


def scale_band(band):
    """Scale band data to 0-255."""
    band_min = band.min()
    band_max = band.max()
    # Éviter la division par zéro
    if band_min == band_max:
        return np.zeros_like(band, dtype=np.uint8)
    return ((band - band_min) / (band_max - band_min) * 255).astype(np.uint8)


class MultiModalDataset(Dataset):
    def __init__(self, directory, data, le, transform=None):
        """
        Initialize the MultiModalDataset.

        Args:
            directory (str): Base directory for image paths
            data (pd.DataFrame): DataFrame containing the dataset information
            le (LabelEncoder): Label encoder for categories
            transform (callable, optional): Optional transform to be applied to images
        """
        self.directory = directory
        self.transform = transform
        self.le = le  # Removed trailing comma that caused a tuple
        self.data = data
        self.feature_columns = [
            'CropCoveredArea', 'CHeight', 'IrriCount', 'WaterCov', 'ExpYield',
            'ndvi', 'evi', 'ndwi', 'gndvi', 'savi', 'msavi', 'SDate_year', 'SDate_month',
            'SDate_day', 'HDate_year', 'HDate_month', 'HDate_day'
        ]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        """
        Get an item from the dataset.

        Args:
            idx (int): Index of the item to get

        Returns:
            tuple: (image, numerical_features, label)
        """
        row = self.data.iloc[idx]
        # Utilise self.directory au lieu de "../data"
        img_path = os.path.join(self.directory, row["tif_path"])

        # Extraire les caractéristiques numériques
        numerical = torch.tensor(
            [row[col] for col in self.feature_columns], dtype=torch.float32)

        # Transformer l'étiquette
        # Ajout des crochets pour éviter les erreurs
        label = self.le.transform([row['category']])[0]

        # Charger et prétraiter l'image
        try:
            with rasterio.open(img_path) as src:

                red = src.read(3)
                green = src.read(2)
                blue = src.read(1)

                rgb = np.dstack((
                    scale_band(red),
                    scale_band(green),
                    scale_band(blue)
                ))

                p2, p98 = np.percentile(rgb, (2, 98))
                rgb = exposure.rescale_intensity(rgb, in_range=(p2, p98))
                rgb = exposure.adjust_gamma(rgb, gamma=0.6)

                image = Image.fromarray(rgb)

                if self.transform:
                    image = self.transform(image)

                return image, numerical, label

        except Exception as e:
            raise RuntimeError(f"Erreur lors du chargement de l'image {
                               img_path}: {str(e)}")

In [46]:
le = LabelEncoder()
y = le.fit(data["category"])
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomRotation(degrees=30),
    transforms.RandomAffine(translate=(0.1, 0.1), degrees=20),
    transforms.ToTensor()

])


train_dataset = MultiModalDataset(
    directory="../data", data=data_clean, le=le, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = MultiModalDataset(
    directory="../data", le=le, data=data_clean, transform=transform)
test_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [56]:
class MultiModalModel(nn.Module):
    def __init__(self, num_numerical_features):
        super(MultiModalModel, self).__init__()
        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = nn.Linear(self.cnn.fc.in_features, 128)

        self.fc_numerical = nn.Sequential(
            nn.Linear(num_numerical_features, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16)
        )

        self.fc_combined = nn.Sequential(
            nn.Linear(128 + 16, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 4),
            # nn.Softmax(dim=1)
            nn.LogSoftmax(dim=1)
        )

    def forward(self, images, numerical):
        image_features = self.cnn(images)
        numerical_features = self.fc_numerical(numerical)
        combined = torch.cat((image_features, numerical_features), dim=1)
        output = self.fc_combined(combined)
        return output

In [31]:
multiModalModel = MultiModalModel(num_numerical_features=17).to(device)

/home/paul/.cache/pypoetry/virtualenvs/telangana-crop-health-challenge-hF4Pj6u_-py3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/paul/.cache/pypoetry/virtualenvs/telangana-crop-health-challenge-hF4Pj6u_-py3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [68]:
def train_dataset(train_loader, num_epochs=10, learning_rate=0.003):
    criterion = nn.NLLLoss(reduction='sum')
    optimizer = torch.optim.Adam(
        multiModalModel.parameters(), lr=learning_rate)
    train_loss = []
    test_loss = []

    for epoch in range(num_epochs):
        total_loss = 0
        for batch_idx, data in enumerate(train_loader):
            images, numerical, labels = data[0], data[1], data[2]
            multiModalModel.train()
            images = images.to(device)
            numerical = numerical.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            output = multiModalModel(images, numerical)
            output = output.float()
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        else:
            multiModalModel.eval()
            total_loss_test = 0
            test_correct = 0 
            for batch_idx, data in enumerate(test_dataloader):
                with torch.no_grad():
                    images, numerical, labels = data[0], data[1], data[2]
                    images = images.to(device)
                    numerical = numerical.to(device)
                    labels = labels.to(device)
                    output = multiModalModel(images, numerical)
                    
                    log_ps = output.float()
                    loss = criterion(output, labels)
                    
                    ps = torch.exp(log_ps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    test_correct += equals.sum().item()                    
                    total_loss_test += loss.item()
        avg_loss_test = total_loss_test / len(test_dataloader)
        avg_loss = total_loss / len(train_loader)
        train_loss.append(avg_loss)
        test_loss.append(avg_loss_test)
        test_correct = test_correct / len(test_dataloader)
        print(f'Epoch {epoch}: Average Loss = {avg_loss:.4f}',
              f'Average Loss Test = {avg_loss_test:.4f}', f'test correct loss test = {test_correct}')

    return train_loss, test_loss

In [69]:
multiModalModel, train_loss, test_loss = train_dataset(train_dataloader)

Epoch 0: Average Loss = -26.4140 Average Loss Test = -26.4140 test correct loss test = 26.413953488372094
Epoch 1: Average Loss = -26.4140 Average Loss Test = -26.4140 test correct loss test = 26.413953488372094
Epoch 2: Average Loss = -26.4140 Average Loss Test = -26.4140 test correct loss test = 26.413953488372094
Epoch 3: Average Loss = -26.4140 Average Loss Test = -26.4140 test correct loss test = 26.413953488372094
Epoch 4: Average Loss = -26.4140 Average Loss Test = -26.4140 test correct loss test = 26.413953488372094


KeyboardInterrupt: 